In [ ]:
from vncorenlp import VnCoreNLP

import numpy as np


In [ ]:
paras = []
min_length = 5
with open("Dataset/wiki_vn/dataset/viwik19_aa") as f:
   while True:
        l = f.readline()
        if not l:
            break
        if l[0] == '=':
            continue
        l = l[:-1]
        paras.append(l)

In [ ]:
len(paras)

In [7]:
paras[0:10]

['Trang Chính',
 'Internet Society',
 'Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.',
 'Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.',
 'Nhiệm vụ và mục đích hoạt động',
 'Bảo đảm, cổ vũ cho sự phát triển, mở rộng và sử dụng Internet được thuận lợi nhất cho mọi người trên toàn thế giới.',
 'Xem thêm',
 '- Lịch sử Internet',
 'Liên kết ngoài',
 '- ISOC Việt Nam']

In [ ]:
annotator = VnCoreNLP("/code/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')


In [9]:
annotator.tokenize(paras[9])

[['-', 'ISOC', 'Việt_Nam']]

In [ ]:
class Vocabulary(object):
    def __init__(self, pad_token='<pad>', unk_token='<unk>', eos_token='<eos>'):
        self.token2idx = {}
        self.idx2token = []
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_token = eos_token
        if pad_token is not None:
            self.pad_index = self.add_token(pad_token)
        if unk_token is not None:
            self.unk_index = self.add_token(unk_token)
        if eos_token is not None:
            self.eos_index = self.add_token(eos_token)

    def add_token(self, token):
        if token not in self.token2idx:
            self.idx2token.append(token)
            self.token2idx[token] = len(self.idx2token) - 1
        return self.token2idx[token]

    def get_index(self, token):
        if isinstance(token, str):
            return self.token2idx.get(token, self.unk_index)
        else:
            return [self.token2idx.get(t, self.unk_index) for t in token]

    def __len__(self):
        return len(self.idx2token)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.__dict__.update(pickle.load(f))

In [ ]:
class Punctuation:
    html = re.compile(r'&apos;|&quot;')
    punctuation = re.compile(r'[^\w\s·]|_')
    spaces = re.compile(r'\s+')
    ela_geminada = re.compile(r'l · l')

    def strip(self, s):
        '''
        Remove all punctuation characters.
        '''
        s = self.html.sub(' ', s)
        s = self.punctuation.sub(' ', s)
        s = self.spaces.sub(' ', s).strip()
        s = self.ela_geminada.sub('l·l', s)
        return s